In [1]:
!pip install -q langchain langchain-core langchain-groq langchain-community yfinance google-search-results

In [2]:
from langchain_groq import ChatGroq

chat = ChatGroq(
    temperature=0,
    model="Llama3-70b-8192",
    api_key="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" # Optional if not set as an environment variable
)
SERP_API="2fd78208042b7ea98ff323aee5d49379d5fb9d29"

In [3]:
import yfinance as yf
import re
import json
from langchain_core.prompts import ChatPromptTemplate

ticker_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
           "From the query from the user provide only the company name and its corresponding ticker symbol in NSE in json format. Dont provide any text other than that?\
            Query : {query}"
        ),
        ("user", "{query}"),
    ]
)
def get_stock_ticker(query):
    chain=ticker_prompt|chat
    return json.loads(chain.invoke({"query":query}).content)

def get_stock_price(ticker,history=15):
    if "." in ticker:
        ticker=ticker.split(".")[0]
    ticker=ticker+".NS"
    stock = yf.Ticker(ticker)
    df = stock.history(period="1y")
    df=df[["Close","Volume"]]
    df.index=[str(x).split()[0] for x in list(df.index)]
    df.index.rename("Date",inplace=True)
    df=df[-history:]
    return df.to_string()


def get_financial_statements(ticker):
    if "." in ticker:
        ticker=ticker.split(".")[0]
    else:
        ticker=ticker
    ticker=ticker+".NS"    
    company = yf.Ticker(ticker)
    balance_sheet = company.balance_sheet
    if balance_sheet.shape[1]>=3:
        balance_sheet=balance_sheet.iloc[:,:3]    # Remove 4th years data
    balance_sheet=balance_sheet.dropna(how="any")
    balance_sheet = balance_sheet.to_string()
    return balance_sheet



In [4]:
from langchain.tools import StructuredTool

get_stockprice_tool = StructuredTool.from_function(func=get_stock_price,name="Get Stock Price",
                                      description="Useful to get the stock price of a company")
get_financialstatements_tool = StructuredTool.from_function(func=get_financial_statements,name="Get Financial Statements",
                                      description="Useful to get the financial statements of a company")
from langchain_community.utilities import GoogleSerperAPIWrapper
news_search=GoogleSerperAPIWrapper(serper_api_key="2fd78208042b7ea98ff323aee5d49379d5fb9d29",type="news")
newssearch_tool = StructuredTool.from_function(
        name="Company News",
        func=news_search.run,
        description="Useful to get the latest news of a company"
    )
tickersearch_tool = StructuredTool.from_function(
        name="Ticker Symbol",
        func=get_stock_ticker,
        description="Use this to get the ticker symbol of the company from the internet"
    )

In [5]:
tools=[tickersearch_tool,newssearch_tool,get_stockprice_tool,get_financialstatements_tool]

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Give detail stock analysis, Use the available data and provide investment recommendation. \
             The user is fully aware about the investment risk, dont include any kind of warning like 'It is recommended to conduct further research and analysis or consult with a financial advisor before making an investment decision' in the answer \
             User question: {query} \
             Fetch the ticker symbol of the company first. Collect all the necessary information and then proceed with analysis. Write (5-8) pointwise investment analysis to answer user query, At the end conclude with proper explaination.Try to Give positives and negatives  : \
               "
        ),
        ("user", "{query}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [7]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
agent = create_tool_calling_agent(chat, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"query": "Is it a good time to invest in Reliance"})



> Entering new AgentExecutor chain...

Invoking: `Ticker Symbol` with `{'query': 'Reliance'}`


{'company': 'Reliance Industries Limited', 'ticker': 'RELIANCE'}
Invoking: `Get Financial Statements` with `{'ticker': 'RELIANCE'}`


                                                           2024-03-31        2023-03-31        2022-03-31
Ordinary Shares Number                                   6766094014.0      6766094014.0      6765994014.0
Share Issued                                             6766094014.0      6766094014.0      6765994014.0
Net Debt                                              2273970000000.0   2400000000000.0   2327960000000.0
Total Debt                                            3461420000000.0   3351340000000.0   2819740000000.0
Tangible Book Value                                   4331070000000.0   4919100000000.0   5476870000000.0
Invested Capital                                     11181030000000.0  11358610000000.0  10457900000000.0
Working Capital           

{'query': 'Is it a good time to invest in Reliance',
 'output': "Based on the financial statements and stock price data, here is a 7-point investment analysis for Reliance Industries Limited:\n\n**Investment Analysis:**\n\n1. **Financial Health:** Reliance Industries Limited has a strong financial position, with a significant amount of cash and cash equivalents (₹972.25 billion) and a low debt-to-equity ratio (0.43). This indicates that the company has a solid foundation to weather any economic downturns.\n\n2. **Revenue Growth:** The company has consistently reported revenue growth over the years, indicating a strong business model and market demand for its products.\n\n3. **Return on Equity (ROE):** Reliance Industries Limited has a high ROE of 15.34%, indicating that the company is generating strong profits from its shareholders' equity.\n\n4. **Valuation:** The company's stock is currently trading at a price-to-earnings (P/E) ratio of 24.51, which is slightly higher than the indust